In [0]:
# Colab 相关设置项
# Mount Google Drive
from google.colab import drive # import drive from google colab

ROOT = "/content/drive"     # default location for the drive
drive.mount(ROOT)           # we mount the google drive at /content/drive
# change to clrs directionary
%cd "/content/drive/My Drive/Colab Notebooks/CLRS/CLRS_notes"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/Colab Notebooks/CLRS/CLRS_notes


In [0]:
%mkdir ch14
!touch ch14/__init__.py

mkdir: cannot create directory ‘ch14’: File exists


In [0]:
import collections
import random
import imp

## 14.1 动态顺序统计

- 第 9 章中的算法可以在 $O(n)$ 的时间内确定任何顺序统计量
- 通过修改红黑树，可以在 $O(\lg n)$ 的时间内确定任何顺序统计量，同时还可在 $O(\lg n)$ 的时间内确
定一个元素的秩，即元素在集合线性序中的位置
- **顺序统计树** $T$ 是在每个结点上附加额外信息的一棵红黑树
  - 附加的额外信息为 $x.size$， 这个属性包含了以 $x$ 为根的子树（包括 $x$ 本身）的结点数，即子树的
  大小，定义哨兵的 $size$ 为0，可得：
    - $$x.size = x.left.size + x.right.size + 1$$
  - 示意图
    - <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200306083856.png width=800>
  - 在顺序统计树中，并不要求关键字互异，因此将元素的秩定义为中序遍历树时输出的位置



###### python 基础代码

In [0]:
%%writefile ch14/os_tree.py
"""顺序统计树"""

from ch13.rb_tree import RBNode, RBTree


class OSTreeNode(RBNode):
  """顺序统计树的结点"""
  def __init__(self, size=0, **kwargs):
    self.size = size
    super().__init__(**kwargs)


class OSTree(RBTree):
  """顺序统计树"""
  def __init__(self, root = None, keys=None):
    self.NIL = OSTreeNode(color=OSTreeNode.B)
    self.root = root if root != None else self.NIL
    if keys:
      for key in keys:
        self.insert(OSTreeNode(key=key))

  def verify(self):
    """验证是否满足顺序统计树的性质"""
    def _helper(x):
      if x != None:
        assert(x.size == x.left.size + x.right.size + 1)
        _helper(x.left)
        _helper(x.right)

    try:
      _helper(self.root)
    except AssertionError:
      raise Exception("Not a order statistics tree!")

Overwriting ch14/os_tree.py


### 对子树规模的维护

#### 插入结点

- 插入结点时，一是要维护红黑树的性质，另外要维护 $size$ 属性

In [0]:
%%writefile -a ch14/os_tree.py


  def insert(self, z):
    x = self.root
    y = self.NIL
    while x != self.NIL:
      y = x
      x.size += 1  # 插入路径上的结点 size 加 1
      if z.key < x.key:
        x = x.left
      else:
        x = x.right
    z.parent = y
    if y == self.NIL:
      self.root = z
    elif z.key < y.key:
      y.left = z
    else:
      y.right = z
    z.left = self.NIL
    z.right = self.NIL
    z.size = 1
    z.color = OSTreeNode.R     
    self.insert_fixup(z)

Appending to ch14/os_tree.py


- 旋转操作时，需要维护相应结点的 `size` 属性
  - <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200306122403.png width=800>
- 旋转操作不会影响最终根节点的的 `size` 属性，因此是局部更新 

In [0]:
%%writefile -a ch14/os_tree.py


  def left_rotate(self, x):
    super(OSTree, self).left_rotate(x)
    x.parent.size = x.size
    x.size = x.left.size + x.right.size + 1
    
  def right_rotate(self, x):
    super(OSTree, self).right_rotate(x)
    x.parent.size = x.size
    x.size = x.left.size + x.right.size + 1


Appending to ch14/os_tree.py


In [0]:
import ch14.os_tree
from ch14.os_tree import OSTree, OSTreeNode

In [0]:
ost = OSTree()
for i in [26, 17, 41, 14, 21, 30, 47, 10, 16, 19, 21, 28, 38, 7, 12, 14, 20, 35, 39, 1]:
  ost.insert(OSTreeNode(key=i))
  ost.verify()

In [0]:
ost


|  |  47

|  41

|  |  |  |  39

|  |  |  38

|  |  |  |  35

|  |  30

|  |  |  28

26

|  |  |  21

|  |  21

|  |  |  |  20

|  |  |  19

|  17

|  |  |  16

|  |  |  |  14

|  |  14

|  |  |  |  12

|  |  |  10

|  |  |  |  7

|  |  |  |  |  1

In [0]:
ost.root.size

20

#### 删除结点

- 设 $y$ 为被删除的结点，或者被移动的后继结点，则需要更新从 $y$ 到根节点的路径上的所有结点，将其
$size$ 减 $1$ 
  - 如果 $x.parent$ 为 $y$ 被删除或移动前的位置，则更新 $x$ 至其父节点的简单路径上的 $size$ 属性
  即可
- 同时需要通过旋转操作，来确保红黑树的性质

In [0]:
%%writefile -a ch14/os_tree.py


  def delete(self, z): 
    x, _, y_origin_color = super(OSTree, self).delete(z, fix=False)
    x1 = x
    while x1.parent != None:
      x1 = x1.parent
      x1.size = x1.left.size + x1.right.size + 1
    if y_origin_color == OSTreeNode.B:
      self.delete_fixup(x)

Appending to ch14/os_tree.py


In [0]:
imp.reload(ch14.os_tree)
from ch14.os_tree import OSTree, OSTreeNode  # 重新加载相应的模块

###### 测试

In [0]:
ost = OSTree()
keys = [26, 17, 41, 14, 21, 30, 47, 10, 16, 19, 21, 28, 38, 7, 12, 14, 20, 35, 39, 1]
for i in keys:
  ost.insert(OSTreeNode(key=i))
  ost.verify()

random.shuffle(keys)
print("原始顺序统计树为：\n{}".format(ost))
for i in keys:
  ost.delete(ost.search(ost.root, i))
  print("删除关键字为 {} 的结点后， 顺序统计树为\n{}".format(i, ost))
  ost.verify()

原始顺序统计树为：

|  |  47

|  41

|  |  |  |  39

|  |  |  38

|  |  |  |  35

|  |  30

|  |  |  28

26

|  |  |  21

|  |  21

|  |  |  |  20

|  |  |  19

|  17

|  |  |  16

|  |  |  |  14

|  |  14

|  |  |  |  12

|  |  |  10

|  |  |  |  7

|  |  |  |  |  1

删除关键字为 28 的结点后， 顺序统计树为

|  |  47

|  41

|  |  |  39

|  |  38

|  |  |  |  35

|  |  |  30

26

|  |  |  21

|  |  21

|  |  |  |  20

|  |  |  19

|  17

|  |  |  16

|  |  |  |  14

|  |  14

|  |  |  |  12

|  |  |  10

|  |  |  |  7

|  |  |  |  |  1

删除关键字为 21 的结点后， 顺序统计树为

|  |  47

|  41

|  |  |  39

|  |  38

|  |  |  |  35

|  |  |  30

26

|  |  |  21

|  |  20

|  |  |  19

|  17

|  |  |  16

|  |  |  |  14

|  |  14

|  |  |  |  12

|  |  |  10

|  |  |  |  7

|  |  |  |  |  1

删除关键字为 1 的结点后， 顺序统计树为

|  |  47

|  41

|  |  |  39

|  |  38

|  |  |  |  35

|  |  |  30

26

|  |  |  21

|  |  20

|  |  |  19

|  17

|  |  |  16

|  |  |  |  14

|  |  14

|  |  |  |  12

|  |  |  10

|  |  |  |  7

删除关键字为 12 的结点后， 顺序统计

### 查找具有给定秩的元素

In [0]:
%%writefile -a ch14/os_tree.py


  def select(self, i):
    """返回第 i 个顺序量"""
    def _helper(x, i):
      if x == self.NIL:
        return x
      r =  x.left.size + 1
      if r == i:
        return x
      if r > i:
        return _helper(x.left, i)
      else:
        return _helper(x.right, i-r)
    return _helper(self.root, i)

Appending to ch14/os_tree.py


In [0]:
imp.reload(ch14.os_tree)
from ch14.os_tree import OSTree, OSTreeNode  # 重新加载相应的模块

In [0]:
ost = OSTree()
for i in [26, 17, 41, 14, 21, 30, 47, 10, 16, 19, 21, 28, 38, 7, 12, 14, 20, 35, 39, 1]:
  ost.insert(OSTreeNode(key=i))
  ost.verify()
ost.select(17)

38

- 每次迭代沿树下降一层，时间复杂度为 $O(\lg n)$

### 确定一个元素的秩

In [0]:
%%writefile -a ch14/os_tree.py


  def rank(self, x):
    """确定一个元素在中序遍历对应的线性序中的位置""" 
    r = x.left.size + 1
    y = x
    while y != self.root:
      if y.parent.right == y:
        r += y.parent.left.size + 1
      y = y.parent
    return r

Appending to ch14/os_tree.py


In [0]:
imp.reload(ch14.os_tree)
from ch14.os_tree import OSTree, OSTreeNode  # 重新加载相应的模块

In [0]:
ost = OSTree()
for i in [26, 17, 41, 14, 21, 30, 47, 10, 16, 19, 21, 28, 38, 7, 12, 14, 20, 35, 39, 1]:
  if i != 38:
    ost.insert(OSTreeNode(key=i))
  else:
    x = OSTreeNode(key=i)
    ost.insert(x)

In [0]:
ost.rank(x)

17

- 每次迭代会沿树上升一层，时间复杂度为 $O(\lg n)$

### 练习

#### 14.1-1, 14.1-2
- <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200309143321.png width=800>
- <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200306083856.png width=800>

- 14.1-1
  - $26: r = 13, i = 10$, go left
  - $17: r = 8, i = 10$, go right
  - $21: r = 3, i = 2$, go left
  - $19: r = 1, i = 2$, go right
  - $20: r = 1, i = 1$, return 20
- 14.1-2
  - $35: r = 1$
  - $38: r = 1$
  - $30: r = r + 1 + 1 = 3$
  - $41: r = 3$
  - $26: r = r + 12 + 1 = 16$ return 16


#### 14.1-3
- <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200309144240.png width=400>

In [0]:
def os_select_iteration(T, i):
  x = T.root
  while x != None:
    r = x.left.size + 1
    if r == i:
      return x
    elif i < r:
      x = x.left
    else:
      x = x.right
      i = i - r

In [0]:
ost = OSTree()
for i in [26, 17, 41, 14, 21, 30, 47, 10, 16, 19, 21, 28, 38, 7, 12, 14, 20, 35, 39, 1]:
  ost.insert(OSTreeNode(key=i))
ost


|  |  47

|  41

|  |  |  |  39

|  |  |  38

|  |  |  |  35

|  |  30

|  |  |  28

26

|  |  |  21

|  |  21

|  |  |  |  20

|  |  |  19

|  17

|  |  |  16

|  |  |  |  14

|  |  14

|  |  |  |  12

|  |  |  10

|  |  |  |  7

|  |  |  |  |  1

In [0]:
os_select_iteration(ost, 17)

38

#### 14.1-4
- <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200309150114.png width=800>

In [0]:
def os_key_rank(x, k):
  """查找关键 k 在以 x 为根的树中的秩，要求 x 中一定存在关键字为 k 的结点"""
  if k == x.key:
    return 1
  elif k < x.key:
    return 1 + os_key_rank(x.left, k)
  else:
    return 1 + x.left.size + os_key_rank(x.right, k)
    

In [0]:
ost = OSTree()
for i in [26, 17, 41, 14, 21, 30, 47, 10, 16, 19, 28, 38, 7, 12, 20, 35, 39, 1]:
  ost.insert(OSTreeNode(key=i))
ost


|  |  47

|  41

|  |  |  |  39

|  |  |  38

|  |  |  |  35

|  |  30

|  |  |  28

26

|  |  |  |  21

|  |  |  20

|  |  |  |  19

|  |  17

|  |  |  16

|  14

|  |  |  12

|  |  10

|  |  |  7

|  |  |  |  1

In [0]:
os_key_rank(ost.root, 38)

15

#### 14.1-5 
- <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200309151249.png width=800>

- 先通过 $OS-RANK(T, x)$ 确定 $x$ 的秩 $j$，然后再通过 $OS-SELECT(T, i+j)$ 确定元素 $x$ 的第 $i$ 个后继

#### 14.1-6
- <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200309151543.png width=800>

- 插入时，设插入的结点为 $z$，对从 $z$ 到根结点的简单路径上的所有结点，如果 $z$ 在该结点的左子树中，则将该结点的秩加 $1$
- 删除的结点设为 $z$，设 $x.parent$ 指向被删除或移动的结点$y$，则对于从 $x$ 到根节点的简单路径上的所有结点，如果 $x$ 在该节点的左子树中，则将该结点的秩减 1。如果 $y$ 是 $z$ 的后继，需要将 $y$ 的秩更新为 $z$ 的秩
- 左旋操作
  - 设对 $x$ 进行左旋操作， 则 $x.left$ 的秩不需要改变，而 $x$ 的秩需要减去 $x.left$ 的秩
- 右旋操作
  - 设对 $y$ 进行右旋操作，则 $y$ 的秩不需要改变，而 $y.left$ 的秩需要加上 $y$ 的秩

##### 代码实现

In [0]:
%%writefile ch14/os_rank_tree.py
""""""
from ch13.rb_tree import RBNode, RBTree


class OSRankTreeNode(RBNode):
  def __init__(self, rank=0, **kwargs):
    """额外含有 rank 属性的红黑树结点"""
    self.rank = rank
    super().__init__(**kwargs)
    
class OSRankTree(RBTree):
  """额外信息为结点在子树中的秩的顺序统计树"""
  def __init__(self, root=None):
    self.NIL = OSRankTreeNode(rank=0, key=None, color=OSRankTreeNode.B)
    self.root = self.NIL if root is None else root

  def left_rotate(self, x):
    """"""
    super().left_rotate(x)   
    x.parent.rank += x.rank
  
  def right_rotate(self, x):
    """"""
    super().right_rotate(x)
    x.rank -= x.parent.rank

  def insert(self, z):
    """"""
    y = self.NIL
    x = self.root
    while x != None:
      y = x
      if z.key < x.key:
        x.rank += 1
        x = x.left
      else:
        x = x.right
    z.parent = y
    if y == None:
      self.root = z
    elif z.key < y.key:
      y.left = z
    else:
      y.right = z
    z.left = self.NIL
    z.right = self.NIL
    z.rank = 1
    z.color = OSRankTreeNode.R
    self.insert_fixup(z)

  def delete(self, z):
    """"""
    x, y, y_origin_color = super().delete(z, fix=False)
      
    x1 = x
    while x1.parent != None:
      if z.key < x1.parent.key:  # 用 key 进行判断，避免边界情况出问题
        x1.parent.rank -= 1
      x1 = x1.parent
    if y != z:  # 需要放在后面运行，防止上面的 while 循环的修改
      y.rank = z.rank
    if y_origin_color == OSRankTreeNode.B:
      self.delete_fixup(x)
  
  def verify(self):
    """验证 rank 属性是否满足要求"""
    def size(root):
      """统计一棵树的大小"""
      if root == None:
        return 0
      return 1 + size(root.left) + size(root.right)
    
    def helper(root):
      """辅助函数"""
      if root == None:
        return
      left_size = size(root.left)
      try:
        assert(root.rank == left_size + 1)
      except AssertionError:
        raise Exception("Not an OSRankTree!")
      helper(root.left)
      helper(root.right)

    helper(self.root)

Overwriting ch14/os_rank_tree.py


###### 测试插入结点

In [0]:
from ch14.os_rank_tree import OSRankTree, OSRankTreeNode

In [0]:
osrt = OSRankTree()
keys = [26, 17, 41, 14, 21, 30, 47, 10, 16, 19, 21, 28, 38, 7, 12, 14, 20, 35, 39, 1]
for i in keys:
  osrt.insert(OSRankTreeNode(key=i))
  osrt.verify()
osrt


|  |  47

|  41

|  |  |  |  39

|  |  |  38

|  |  |  |  35

|  |  30

|  |  |  28

26

|  |  |  21

|  |  21

|  |  |  |  20

|  |  |  19

|  17

|  |  |  16

|  |  |  |  14

|  |  14

|  |  |  |  12

|  |  |  10

|  |  |  |  7

|  |  |  |  |  1

###### 测试删除结点

In [0]:
osrt = OSRankTree()
keys = [26, 17, 41, 14, 21, 30, 47, 10, 16, 19, 21, 28, 38, 7, 12, 14, 20, 35, 39, 1]
for i in keys:
  osrt.insert(OSTreeNode(key=i))

random.shuffle(keys)
print("原始顺序统计树为：\n{}".format(osrt))
for i in keys:
  osrt.delete(osrt.search(osrt.root, i))
  print("删除关键字为 {} 的结点后， 顺序统计树为\n{}".format(i, osrt))
  osrt.verify()

原始顺序统计树为：

|  |  47

|  41

|  |  |  |  39

|  |  |  38

|  |  |  |  35

|  |  30

|  |  |  28

26

|  |  |  21

|  |  21

|  |  |  |  20

|  |  |  19

|  17

|  |  |  16

|  |  |  |  14

|  |  14

|  |  |  |  12

|  |  |  10

|  |  |  |  7

|  |  |  |  |  1

删除关键字为 21 的结点后， 顺序统计树为

|  |  47

|  41

|  |  |  |  39

|  |  |  38

|  |  |  |  35

|  |  30

|  |  |  28

26

|  |  |  21

|  |  20

|  |  |  19

|  17

|  |  |  16

|  |  |  |  14

|  |  14

|  |  |  |  12

|  |  |  10

|  |  |  |  7

|  |  |  |  |  1

删除关键字为 47 的结点后， 顺序统计树为

|  |  |  41

|  |  |  |  39

|  |  38

|  |  |  35

|  30

|  |  28

26

|  |  |  21

|  |  20

|  |  |  19

|  17

|  |  |  16

|  |  |  |  14

|  |  14

|  |  |  |  12

|  |  |  10

|  |  |  |  7

|  |  |  |  |  1

删除关键字为 39 的结点后， 顺序统计树为

|  |  |  41

|  |  38

|  |  |  35

|  30

|  |  28

26

|  |  |  21

|  |  20

|  |  |  19

|  17

|  |  |  16

|  |  |  |  14

|  |  14

|  |  |  |  12

|  |  |  10

|  |  |  |  7

|  |  |  |  |  1

删除关键字为 35 的结点后， 顺

#### 14.1-7
- <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200309173359.png width=800>

- 依次将数组中的元素插入顺序统计树中，并通过 OS-RANK 确定所插入元素的秩 $x$, 设元素在数组中的的下标
（起始下标为 $1$）为 $y$，则由于插入此元素所引入的逆序对数为 $y-x$
- 每插入一个元素和计算其秩，时间复杂度为 $O(\lg n)$， 插入 $n$ 个元素，其复杂度即为 $O(n\lg n)$

###### 代码实现

In [0]:
def inversions(arr):
  ost = OSTree()
  sum = 0
  for index, item in enumerate(arr, 1):
    node = OSTreeNode(key=item)
    ost.insert(node)
    sum += index - ost.rank(node)
  return sum

In [0]:
arr = random.sample(range(1, 100), 10)
print("数组为: ", arr)
print("逆序对为: ", inversions(arr))

数组为:  [72, 71, 45, 19, 15, 98, 28, 16, 29, 68]
逆序对为:  27


#### 14.1-8
- <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200309185043.png width=800>

- 将 $2n$ 个端点按照顺时针方向排列，然后按照从小到大的顺序给各个端点赋予独特的值，将每条弦中值较小的端点
提取出来，组成集合 $x$，然后值较大的端点组成集合 $y$， $x$, $y$ 组成集合 $z$。 $z$ 中的元素
按顺序排列    
- 按顺序遍历 $z$ 中的元素，设第 $i$ 次遍历元素值为$z_i$
  - 如果 $z_i \in x$，则将其插入顺序统计树 $T$ 中，并将树中总的元素数目$s$ 加 1  
  - 如果 $z_i \in y$，则需要找出以 $z_i$ 为端点的弦的另一个端点 $x_i$，然后计算 $x_i$ 的秩，设为$j$，
则一共有 $s - j$ 条弦与以 $x_i, z_i$ 为端点的弦相交。 然后将 $x_i$ 从 $T$ 中删除，并将 $s$ 减
$1$
- 遍历完 $z$ 中的元素，即可确定圆内相交弦的对数

## 14.2 如何扩张数据结构

- 对基本的数据结构进行扩张以支持一些附加功能，在算法设计过程中很常见
- 扩张一种数据结构基本需要包括以下 4 个步骤
  1. 选择一种基础数据结构
  2. 确定基础数据结构中需要维护的附加信息
  3. 检验基础数据结构上的基本修改操作能否维护附加信息
  4. 设计一些新操作

### 红黑树的扩张

###### 定理 14.1
- 设 $f$ 是 $n$ 个结点的红黑树的 $T$ 的扩张属性，且假设对任一结点 $x$， $f$ 的值仅依赖结点 $x, x.left, x.right$ 的信
息。那么，可以在插入或删除操作期间对 $T$ 的所有结点的 $f$ 值进行维护，并且不影响这两个操作的 $O(\lg n)$ 渐近时间性能

### 练习

#### 14.2-1
- <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200309194310.png width=800>

- 可以向每个结点中添加 $next$ 和 $prev$ 属性，其中$next$ 指向后继， $prev$ 指向前驱，同时将
$NIL$ 结点做为空头指针，则 $NIL.next$ 指向最小值， $NIL.prev$则指向最大值。相当于各个结点组成了双向
链表，如此查询最值和前驱及后继只需 $O(1)$ 的操作
- 整体维护时相当于维护一个双向链表
  - 维护最值和删除某个结点，均可在 $O(1)$ 的时间完成
  - 插入某个结点时，可以借助 $OS-SELECT$ 和 $OS-RANK$ 在 $O(\lg n)$ 的时间内确定其后继，
  然后将插入的结点在双向链表中插入到其值之前即可

In [0]:
%%writefile ch14/os_tree_with_linkedlist.py
"""带有双向链表的顺序统计树"""

from ch14.os_tree import OSTree, OSTreeNode


class OSTreeWithLinkedListNode(OSTreeNode):
  """带有双向链表的顺序统计树结点"""
  def __init__(self, prev=None, next=None, **kwargs):
    super().__init__(**kwargs)
    self.prev = prev
    self.next = next


class OSTreeWithLinkedList(OSTree):
  """带有双向链表的顺序统计树"""
  def __init__(self, root=None):
    self.NIL = OSTreeWithLinkedListNode(key=None, color=OSTreeWithLinkedListNode.B)
    self.NIL.prev = self.NIL
    self.NIL.next = self.NIL
    self.root = self.NIL if root is None else root
  
  def insert(self, z):
    super().insert(z)
    z.next = self.select(self.rank(z) + 1)
    z.prev = z.next.prev
    z.prev.next = z
    z.next.prev = z
  
  def delete(self, z):
    super().delete(z)
    z.next.prev = z.prev
    z.prev.next = z.next
  
  def mini(self):
    return self.NIL.next
  
  def maxi(self):
    return self.NIL.prev

  def successor(self, z):
    return z.next

  def predecessor(self, z):
    return z.prev
  
  def print_linkedlist(self):
    """打印链表"""
    res = 'None ->'
    x = self.NIL.next
    while x != None:
      res += "{} ->".format(x)
      x = x.next
    res += 'None'
    return res


if __name__ == "__main__":
  import random
  ostwll = OSTreeWithLinkedList()
  keys = random.sample(range(100), 10)
  for key in keys:
    ostwll.insert(OSTreeWithLinkedListNode(key=key))
  print('二叉树为：\n{}'.format(ostwll))
  print("双向链表为：{}".format(ostwll.print_linkedlist()))
  print("最大值为：{}".format(ostwll.maxi()))
  print("最小值为：{}".format(ostwll.mini()))
  i = random.choice(keys)
  print("键为 {} 的前驱为:{}".format(i, ostwll.predecessor(ostwll.search(ostwll.root, i))))
  j = random.choice(keys)
  print("键为 {} 的后继为:{}".format(j, ostwll.successor(ostwll.search(ostwll.root, j))))
  print("- "*50)
  print("测试删除结点：")
  random.shuffle(keys)
  for key in keys:
    ostwll.delete(ostwll.search(ostwll.root, key))
    print("- "*50)
    print("删除键 {} 后，二叉树为: \n{}".format(key, ostwll))
    print("双向链表为：{}".format(ostwll.print_linkedlist()))
    print("最大值为：{}".format(ostwll.maxi()))
    print("最小值为：{}".format(ostwll.mini()))

Overwriting ch14/os_tree_with_linkedlist.py


In [0]:
!python3 -m ch14.os_tree_with_linkedlist

二叉树为：

|  |  |  92

|  |  77

|  |  |  72

|  62

|  |  60

|  |  |  59

55

|  |  51

|  25

|  |  1

双向链表为：None ->1 ->25 ->51 ->55 ->59 ->60 ->62 ->72 ->77 ->92 ->None
最大值为：92
最小值为：1
键为 51 的前驱为:25
键为 92 的后继为:None
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
测试删除结点：
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
删除键 60 后，二叉树为: 

|  |  |  92

|  |  77

|  |  |  72

|  62

|  |  59

55

|  |  51

|  25

|  |  1

双向链表为：None ->1 ->25 ->51 ->55 ->59 ->62 ->72 ->77 ->92 ->None
最大值为：92
最小值为：1
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
删除键 62 后，二叉树为: 

|  |  |  92

|  |  77

|  72

|  |  59

55

|  |  51

|  25

|  |  1

双向链表为：None ->1 ->25 ->51 ->55 ->59 ->72 ->77 ->92 ->None
最大值为：92
最小值为：1
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
删除键 92 后，二叉树为: 

|  |  77

|  72

|  

#### 14.2-2
- <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200310121801.png width=800>


- 黑高可以做为树中的一个结点进行维护，因为一个结点的黑高只与其自身的颜色和其子结点的黑高有关，由定理 14.1 可知，可以在不影响
红黑树操作的渐近性能的前提下，维据结点的黑高属性
  - 插入
    - 插入的结点为红色，不会影响其它黑高
    - INSERT-FIXUP 会改变某些结点的颜色，从而该结点的黑高发生改变，需要进行维护
  - 删除
    - 删除时会加入额外的黑色，不会影响其它结点的黑高。但如果 $y$　为后继，且不是被删除结点的子节点，其黑高可能发生变化
    - DELETE-FIXUP 运行时，会改变某些结点的颜色，需要对其黑高进行维护 
- 结点的深度则无法在渐近时间内进行维护，因为如果删除了根节点，则所有节点的深度均会发生改变，需要 $O(n)$ 的时间

#### 14.2-3
- <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200310130818.png width=800>


- 旋转后，设 $x$ 为深度较深的节点， $y$ 为深度较高的节点， 可将 $y.f$ 设为 $x.f$ 的值，而 $x.f$ 可按下式进行计算：
$$x.f = x.left.f \bigotimes x.a \bigotimes x.right.f$$
- 对于 $size$ 属性， 对于每一个非 NIL 的结点来说， $a$ 始终为 1，而 $\bigotimes$ 运算符即相当于 $+$ 运算符， 由此
便可将相关的性质扩展到 $size$ 属性中。

14.2-4
- <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200310133512.png width=800>


- 如果 $x.key \ge a $ 则一定要向左子树遍历， 如果 $x.key \le b$， 则一定要向右子树遍历。 从根结节遍历至叶子节点，需用时为
$\Theta(\lg n)$， 输出 $m$ 个关键字，需要用时 $\Theta(m)$，则总的算法复杂度为 $\Theta(m + \lg n)$

In [0]:
def rb_enumerate(T, a, b):
  def _helper(x, a, b):
    if x == None:
      return
    if a <= x.key <= b:
      print(x, end=" ")
    if x.key >= a:
      _helper(x.left, a, b)
    if x.key <= b:
      _helper(x.right, a, b)

  _helper(T.root, a, b)

In [0]:
from ch13.rb_tree import RBTree
rbt = RBTree(keys=random.sample(range(100), 10))
print("红黑树为：\n", rbt)

a = 30
b = 70
print("[{}, {}] 间的结点有： ".format(a, b), end="")
rb_enumerate(rbt, a, b)

红黑树为：
 
|  |  93

|  70

|  |  62

58

|  |  |  52

|  |  45

|  |  |  36

|  22

|  |  16

|  |  |  11

[30, 70] 间的结点有： 58 45 36 52 70 62 

## 14.3 区间树

- **闭区间** 指一个实数的有序对 $[t_1, t_2]$， 其中 $t_1 \le t_2$。 **开**区间和**半开**区间分别略去了集合中的两个或一
个端点
- 区间便于表示占用一连续时间段的一些事件
  - 查询一个由时间区间数据构成的数据库，去找出在给一的时间区间内发生了什么事件

###### 区间三分律

- 将 $[t_1, t_2]$ 表示为一个对象 $i$
  - $i.low=t_1$ 为低端点
  - $i.high=t_2$ 为高端点
- 任何两个区间的 $i$ 和 $i^{'}$ 满足区间三分律，即下面三条性质之一必成立
  1. $i$ 和 $i^{'}$ 重叠 ($i.low \le i^{'}.high$ 且 $i^{'}.low < i.high$)
  2. $i$ 在 $i^{'}$ 的左边 ($i.high < i^{'}.low$)
  2. $i$ 在 $i^{'}$ 的右边 ($i^{'}.high < i.low$)
  - 示意图
    - <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200306141549.png width=800>

###### 区间树的基本概念

- **区间树**(interval tree) 是一种对动态集合进行维护的红黑树，每一个元素包含一个区间 $x.int$，除了支持常规的插入，删除外，其
还支持搜索操作
  - $INTERVAL-SEARCH(T, i)$ 返回一个指向区间树 $T$ 中元素 $x$ 的指针， 使得 $x.int$ 与 $i$　重叠，若此元素不存在，则
  返回 $T.nil$
- 区间树示意图
  - <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200306142718.png width=700>

### 区间树的实现

#### 步骤1： 基础数据结构

- 每个结点包含属性 $x.int$，$x$ 的关键字取为区间的低端点 $x.int.low$

In [0]:
%%writefile ch14/interval_tree.py
"""区间树"""
from ch13.rb_tree import RBNode, RBTree
import collections


Interval = collections.namedtuple('Interval', ['low', 'high'])  # 区间类


class IntervalTreeNode(RBNode):
  def __init__(self, low=None, high=None, maxi=None, **kwargs):
    super().__init__(**kwargs)
    self.int = Interval(low, high)
    self.key = self.int.low
    self.maxi = maxi

Overwriting ch14/interval_tree.py


#### 步骤2：附加信息

- 包含 $x.max$，它是以 $x$ 为根的子树中所有区间端点的最大值
  - Python 代码中设置为 $x.maxi$


#### 步骤3： 对信息进行维护

- 主要是对 $x.max$ 进行维护
  - $$x.max = max(x.int.high, x.left.max, x.right.max)$$
- 由定理 14.1 可知，插入和删除操作的运行时间为 $O(\lg n)$

In [0]:
%%writefile -a ch14/interval_tree.py



class IntervalTree(RBTree):
  """区间树"""
  def __init__(self, intervals=None):
    self.NIL = IntervalTreeNode(maxi=float('-inf'),color=IntervalTreeNode.B)
    self.root = self.NIL
    if intervals:
      for interval in intervals:
        self.insert(IntervalTreeNode(low=interval[0], high=interval[1]))
  
  def insert(self, z):
    x = self.root
    y = self.NIL
    while x != self.NIL:
      y = x
      x.maxi = max(x.maxi, z.int.high)
      if z.key < x.key:
        x = x.left
      else:
        x = x.right
    
    z.parent = y
    if y == self.NIL:
      self.root = z
    elif z.key < y.key:
      y.left = z
    else:
      y.right = z
    z.left = self.NIL
    z.right = self.NIL
    z.maxi = z.int.high
    z.color = IntervalTreeNode.R
    self.insert_fixup(z)

  def delete(self, z):
    """在区间树中删除结点 z"""
    x, _, y_origin_color = super().delete(z, fix=False)
    x1 = x
    while x1.parent != None:
      x1 = x1.parent
      x1.maxi = max(x1.int.high, x1.left.maxi, x1.right.maxi)
      
    if y_origin_color == IntervalTreeNode.B:
      self.delete_fixup(x)
  
  def verify(self):
    """验证是 maxi 属性是否符合区间树的要求"""
    def _helper(root):
      if root != None:
        assert( root.maxi == max(root.int.high, root.left.maxi, root.right.maxi))
        _helper(root.left)
        _helper(root.right)
    try:
      _helper(self.root)
    except AssertionError:
      raise Exception("Not a interval tree!")

  def left_rotate(self, x):
    super().left_rotate(x)
    x.parent.maxi = x.maxi
    x.maxi = max(x.left.maxi, x.right.maxi, x.int.high)


  def right_rotate(self, x):
    super().right_rotate(x)
    x.parent.maxi = x.maxi
    x.maxi = max(x.left.maxi, x.right.maxi, x.int.high)


Appending to ch14/interval_tree.py


In [0]:
import ch14.interval_tree
from ch14.interval_tree import IntervalTree, IntervalTreeNode

In [0]:
intervals = [(16, 21), (8, 9), (25, 30), (5, 8), (15, 23), (17, 19), (26, 26), (0, 3), (6, 10), (19, 20)]
interval_nodes = [IntervalTreeNode(i[0], i[1]) for i in intervals]
interval_tree = IntervalTree()
for node in interval_nodes:
  interval_tree.insert(node)
  interval_tree.verify()

random.shuffle(intervals)
print("原始的区间树为：\n{}".format(interval_tree))
for item in intervals:
  interval_tree.delete(interval_tree.search(interval_tree.root, item[0]))
  print("删除区间[ {}, {}] 后, 区间树为： \n{}".format(item[0], item[1], interval_tree))
  interval_tree.verify()

原始的区间树为：

|  |  26

|  25

|  |  |  19

|  |  17

16

|  |  15

|  8

|  |  |  6

|  |  5

|  |  |  0

删除区间[ 0, 3] 后, 区间树为： 

|  |  26

|  25

|  |  |  19

|  |  17

16

|  |  15

|  8

|  |  |  6

|  |  5

删除区间[ 25, 30] 后, 区间树为： 

|  |  26

|  19

|  |  17

16

|  |  15

|  8

|  |  |  6

|  |  5

删除区间[ 26, 26] 后, 区间树为： 

|  19

|  |  17

16

|  |  15

|  8

|  |  |  6

|  |  5

删除区间[ 19, 20] 后, 区间树为： 

|  17

16

|  |  15

|  8

|  |  |  6

|  |  5

删除区间[ 8, 9] 后, 区间树为： 

|  17

16

|  |  15

|  6

|  |  5

删除区间[ 16, 21] 后, 区间树为： 

|  17

|  |  15

6

|  5

删除区间[ 6, 10] 后, 区间树为： 

|  17

15

|  5

删除区间[ 15, 23] 后, 区间树为： 

17

|  5

删除区间[ 17, 19] 后, 区间树为： 

5

删除区间[ 5, 8] 后, 区间树为： 
None


#### 步骤4：设计新的操作

In [0]:
%%writefile -a ch14/interval_tree.py


  def search(self, i):
    """返回一个与 i 重叠的区间，如果没有，则返回 NIL"""
    x = self.root
    while x != self.NIL and (i.low > x.int.high or x.int.low > i.high):
      x = x.left if x.left != self.NIL and i.low <= x.left.maxi else x.right
    return x

Appending to ch14/interval_tree.py


In [0]:
imp.reload(ch14.interval_tree)
from ch14.interval_tree import IntervalTree, IntervalTreeNode, Interval# 重新加载模块

In [0]:
interval_tree = IntervalTree([(16, 21), (8, 9), (25, 30), (5, 8), (15, 23), (17, 19), (26, 26), (0, 3), (6, 10), (19, 20)])
interval = interval_tree.search(Interval(22, 25))
interval.int

Interval(low=15, high=23)

In [0]:
interval = interval_tree.search(Interval(11, 14))
interval

None

- $INTERVAL-SEARCH$ 的工作原理示意图
  - <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200306190418.png width=800>
- 如果往右子树中查找，则左子树中一定没有重叠的区间
- 如果往左子树中查找，如果左子树中不存在重叠的区间，则右子树中一定也不存在重叠的区间

### 练习

#### 14.3-2
  - <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200310143203.png width=800>

- 需要更改区间是否重叠的判断条件， 以及往左右子树查找的判断条件

In [0]:
def open_interval_search(T, i):
  x = T.root
  while x != None and ( i.low >= x.int.high or x.int.low >= i.high):
    x = x.left if x.left != None and i.low < x.left.maxi else x.right
  return x

#### 14.3-3 
- <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200310144144.png width=800>

- 每次找到一个区间后，将 $x$ 置为 $x.left$，然后继续循环，直至 $x$ 为 NIL
- 由于 $INTERVAL-SEARCH$ 只会在左子树不存在重叠区间时，才会往右子树查找，因为可以确保最后一个找到的重叠区间拥有最小的端点

In [0]:
%%writefile -a ch14/interval_tree.py


  def search_minimum(self, i):
    """返回与 i 重叠的，具有最小低端点的区间"""
    x = self.root
    res = self.NIL
    while x != None:
      if x.int.low <= i.high and i.low <= x.int.high:
        res = x
        x = x.left
        continue
      x = x.left if x.left != None and x.left.maxi <= i.low else x.right
    
    return res

Appending to ch14/interval_tree.py


In [0]:
imp.reload(ch14.interval_tree)
from ch14.interval_tree import IntervalTree, IntervalTreeNode, Interval# 重新加载模块

In [0]:
intervals = []
for i in range(10):
  intervals.append(sorted(random.sample(range(100), 2)))
interval_tree = IntervalTree(intervals)
interval_tree


|  |  71

|  65

|  |  51

|  |  |  49

33

|  |  |  32

|  |  14

|  |  |  9

|  6

|  |  2

In [0]:
i = Interval(1, 100)
print(interval_tree.search_minimum(i).int)

Interval(low=2, high=55)


#### 14.3-4
- <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200310152738.png width=800>

- 基本思路与 $14.2-4$ 相同， 如果子树中可能存在相应的结点，则向相应的子树进行遍历

In [0]:
%%writefile -a ch14/interval_tree.py


  def search_all(self, i):
    def _helper(x, i):
      if x == None:
        return
      if i.low <= x.int.high and x.int.low <= i.high:
        print(x.int)
      if x.left != None and x.left.maxi >= i.low:
        _helper(x.left, i)
      if x.right != None and x.right.maxi >= i.low and x.int.low <= i.high:
        _helper(x.right, i)

    _helper(self.root, i)

Appending to ch14/interval_tree.py


In [0]:
imp.reload(ch14.interval_tree)
from ch14.interval_tree import IntervalTree, IntervalTreeNode, Interval# 重新加载模块

In [0]:
intervals = []
for i in range(10):
  intervals.append(sorted(random.sample(range(100), 2)))
interval_tree = IntervalTree(intervals)
interval_tree


|  |  |  53

|  |  44

|  33

|  |  |  30

|  |  19

|  |  |  13

13

|  |  12

|  5

|  |  1

In [0]:
interval_tree.search_all(Interval(45, 55))

Interval(low=12, high=71)
Interval(low=33, high=82)
Interval(low=19, high=66)
Interval(low=13, high=45)
Interval(low=44, high=56)
Interval(low=53, high=63)


#### 14.3-5
- <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200310160753.png width=800>

- 选按低端点进行搜索，如果找到低端点相同的区间，需要判断高端点是否相同，如果不相同，则需要继续向下遍历

In [0]:
%%writefile -a ch14/interval_tree.py


  def search_excatly(self, i):
    """查找区间完全相同的结点"""
    def helper(x, i):
      res = None
      if x.left != None and i.low < x.key and x.left.maxi >= i.high:
        return helper(x.left, i)
      if x.right != None and i.low > x.key and x.right.maxi >= i.high:
        return helper(x.right, i)
      if i.low == x.key:
        if i.high == x.int.high:
          return x
        if x.left != None and x.left.maxi >= i.high:
          res = helper(x.left, i)
        if res != None:
          return res
        elif x.right != None and x.right.maxi >= i.high:
          res = helper(x.right, i)
      return res

    return None if self.root == None else helper(self.root, i)

Appending to ch14/interval_tree.py


In [0]:
imp.reload(ch14.interval_tree)
from ch14.interval_tree import IntervalTree, IntervalTreeNode, Interval# 重新加载模块

In [0]:
intervals = []
for i in range(10):
  intervals.append(sorted(random.sample(range(100), 2)))
interval_tree = IntervalTree(intervals)
interval_tree


|  |  80

|  70

46

|  |  |  36

|  |  32

|  |  |  24

|  19

|  |  |  15

|  |  8

|  |  |  0

In [0]:
interval = random.choice(intervals)
i = Interval(interval[0], interval[1])
print("搜索的区间为： [{}, {}]".format(i.low, i.high))
res = interval_tree.search_excatly(i)
print(res)
print(res.int)

搜索的区间为： [46, 49]
46
Interval(low=46, high=49)


In [0]:
type(res)

ch14.interval_tree.IntervalTreeNode

#### 14.3-6
- <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200310174951.png width=800>

- 每个结点的关键字为为其真实值，额个信息为以当前节点为根的树中的最大值$maxi$，最小值$mini$和 $mini\_gap$
- $NIL$ 结点的最大值为 $- \infty$，最小值为 $+ \infty$， $mini\_gap$ 为 $+ \infty$
- 叶子结点的 $mini\_gap$ 
为 $+\infty$
- 3 个附加属性的维护（二叉搜索树的性质，左子树的关键字不超过根节点，右子树的关键字不小于根结点）
  - $x.maxi = max(x.right.maxi, x.key)$
  - $x.mini = min(x.left.mini, x.key)$
  - $\begin{aligned}
  x.mini\_gap = &min(x.left.mini\_gap, x.right.mini\_gap, x.key - x.left.maxi, \\
  &x.right.mini - x.key)
  \end{aligned}$
- 3 个附加的属性均只与结点自身及其左右结点有关，因此可以在 $O(\lg n)$ 的时间内完成 $INSERT, DELETE,
SEARCH$ 操作， $MIN-GAP$ 只需返回根节点的 $mini\_gap$ 属性即可
***
- 代码实现

In [0]:
%%writefile ch14/rb_tree_with_mini_gap.py
"""带有 mini_gap 属性的红黑树"""

from ch13.rb_tree import RBNode, RBTree


class RBNodeWithMiniGap(RBNode):
  """带有 mini_gap 属性的红黑树结点"""
  def __init__(self, maxi=None, mini=None, mini_gap = None, **kwargs):
    self.maxi = maxi
    self.mini= mini
    self.mini_gap = mini_gap
    super().__init__(**kwargs)
    
    
class RBTreeWithMiniGap(RBTree):
  """带有 mini_gap 属性的红黑树"""
  def __init__(self, root = None, keys=None):
    self.NIL = RBNodeWithMiniGap(maxi=float('-inf'), mini=float('inf'), mini_gap=float('inf'), key=None, color=RBNodeWithMiniGap.B)
    self.root = self.NIL if root == None else root
    if keys:
      for key in keys:
        self.insert(RBNodeWithMiniGap(key=key))

  @staticmethod
  def update(x):
    x.mini = min(x.left.mini, x.key)
    x.maxi = max(x.right.maxi, x.key)
    x.mini_gap = min(x.left.mini_gap, x.right.mini_gap, x.key - x.left.maxi, x.right.mini - x.key)

  def insert(self, z):
    super().insert(z, fix=False)       
    z.maxi = z.key
    z.mini = z.key
    z.mini_gap = float('inf')
    x = z.parent
    while x != None:
      self.update(x)
      x = x.parent
    self.insert_fixup(z)
  
  def delete(self, z):
    x, _, y_origincolor = super().delete(z, fix=False)
    y = x.parent
    while y != None:
      self.update(y)
      y = y.parent
    if y_origincolor == RBNodeWithMiniGap.B:
      self.delete_fixup(x)

  def left_rotate(self, x):
    super().left_rotate(x)
    x.parent.mini = x.mini
    x.parent.maxi = x.maxi
    x.parent.mini_gap = x.mini_gap
    self.update(x)
  
  def right_rotate(self, x):
    super().right_rotate(x)
    x.parent.mini = x.mini
    x.parent.maxi = x.maxi
    x.parent.mini_gap = x.mini_gap
    self.update(x)

  def mini_gap(self):
    return  self.root.mini_gap if self.root.mini_gap != float('inf') else None
    


Overwriting ch14/rb_tree_with_mini_gap.py


In [0]:
from ch14.rb_tree_with_mini_gap import RBTreeWithMiniGap

In [0]:
keys = random.sample(range(100), 10)
rbt_with_minigap = RBTreeWithMiniGap(keys=keys)

In [0]:
print("Q = ", sorted(keys))
print("mingap = ", rbt_with_minigap.mini_gap())

Q =  [1, 22, 25, 31, 37, 42, 43, 58, 91, 98]
mingap =  1


In [0]:
keys = random.sample(range(100), 10)
rbt_with_minigap = RBTreeWithMiniGap(keys=keys)
print("Q = ", sorted(keys))
print("mingap = ", rbt_with_minigap.mini_gap())
print("- " * 10)


random.shuffle(keys)
remain_keys = keys.copy()
for key in keys:
  remain_keys.remove(key)
  print("删除 {} 后， Q = {}".format(key, sorted(remain_keys)))
  rbt_with_minigap.delete(rbt_with_minigap.search(rbt_with_minigap.root, key)) 
  print("mingap = ", rbt_with_minigap.mini_gap())
  print("- " * 10)

Q =  [9, 11, 15, 22, 27, 42, 60, 76, 85, 86]
mingap =  1
- - - - - - - - - - 
删除 9 后， Q = [11, 15, 22, 27, 42, 60, 76, 85, 86]
mingap =  1
- - - - - - - - - - 
删除 42 后， Q = [11, 15, 22, 27, 60, 76, 85, 86]
mingap =  1
- - - - - - - - - - 
删除 60 后， Q = [11, 15, 22, 27, 76, 85, 86]
mingap =  1
- - - - - - - - - - 
删除 85 后， Q = [11, 15, 22, 27, 76, 86]
mingap =  4
- - - - - - - - - - 
删除 11 后， Q = [15, 22, 27, 76, 86]
mingap =  5
- - - - - - - - - - 
删除 27 后， Q = [15, 22, 76, 86]
mingap =  7
- - - - - - - - - - 
删除 76 后， Q = [15, 22, 86]
mingap =  7
- - - - - - - - - - 
删除 15 后， Q = [22, 86]
mingap =  64
- - - - - - - - - - 
删除 86 后， Q = [22]
mingap =  None
- - - - - - - - - - 
删除 22 后， Q = []
mingap =  None
- - - - - - - - - - 


#### 14.3-7
- <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200310175531.png width=800>

- 基本思路是借助一条平行于 $y$ 轴的扫描线，看在此扫描线上，是否有重叠的矩形
- 每一个矩阵可用坐标 $(x_{1i}, x_{2i})$ 和  $(y_{1i}, y_{2i})$ 来表示
  - $i=1 \cdots n$ 表示第 $i$ 个矩阵
  - $x_{1i} \lt x_{2i}, y_{1i} \lt y_{2i}$
- 将 $x$ 按顺序排列组成集合 $X$， 初始化一棵区间树 $T$， 用于存放矩阵 $y$ 轴方向上的区间，然后依次
遍历 $x$ 中的元素
  - 如果碰到第 $i$ 个矩阵较小的 $x$ 轴坐标， 即 $x_{1i}$，则检测区间 $[y_{1i}, y_{2i}]$ 是否在树
  $T$中有重合的区间
    - 如果有，则说明矩阵集合中含有重叠的矩形，可结束函数运行
    - 如果没有，则将区间 $[y_{1i}, y_{2i}]$ 插入到区间树 $T$ 中
  - 如果碰到第 $i$ 个矩阵较大的 $x$ 轴坐标， 即 $x_{2i}$，则将区间 $[y_{1i}, y_{2i}]$ 从区间树
  $T$ 中删除
- 如果遍历完集合 $X$ 中的元素，仍然没有发现重叠的矩形，说明矩阵集合中没有重叠的矩形
***
- 复杂度分析
  - $2n$ 个 $x$ 坐标排序所需时间为 $O(n\lg n)$
  - 每个矩阵需要对区间树 $T$ 分别进行一次 $SEARCH, INSERT, DELETE$ 操作，单次操作的时间复杂度为
$O(\lg n)$，总的时间复杂度为 $O(n\lg n)$
  - 综上，总的时间复杂度为 $O(n\lg n)$

***
- 代码实现

In [0]:
%%writefile ch14/vlsi_overlap_rectangles.py

from ch14.interval_tree import IntervalTreeNode, IntervalTree
import collections


# x 坐标的命名元组，key 指坐标值， isleft 为 True，表明是矩阵的左边，为 False 表示是矩阵的右边， y_interval 为矩阵对应的 y 轴区间结点
XCoordinate = collections.namedtuple('XCoordinate', ['key', 'is_left', 'y_interval']) 


def get_x_coordinates(rectangle):
  """
  返回矩阵的两个 x 轴坐标
  rectangle = (x1, x2, y1, y2), 其中x1 < x2, y1 < y2
  """
  x1, x2, *y_interval = rectangle
  y_interval = IntervalTreeNode(low=y_interval[0], high=y_interval[1])
  x_low = XCoordinate(x1, True, y_interval)
  x_high = XCoordinate(x2, False, y_interval)
  return x_low, x_high
  

def is_overlap(rectangles):
  """
  检测是否有重合的矩阵
  rectangles 为包含 rectangle 的可迭代对象
  rectangle = (x1, x2, y1, y2), 其中x1 < x2, y1 < y2
  """
  x_list = []
  for rectangle in rectangles:
    x_list.extend(get_x_coordinates(rectangle))
  x_list.sort(key=lambda x: x.key)
  y_interval_tree = IntervalTree()
  for x in x_list:
    if x.is_left:
      if y_interval_tree.search(x.y_interval.int) != None:
        return True
      y_interval_tree.insert(x.y_interval)
    else:
      y_interval_tree.delete(x.y_interval)
  return False


if __name__ == "__main__":
  rectangles = [[2, 5, 7, 44], [50, 53, 3, 82], [67, 84, 17, 62], [22, 38, 42, 50], [5, 7, 90, 95]]
  print("矩阵集合为： ", rectangles)
  if is_overlap(rectangles):
    print("存在重合矩阵")
  else:
    print("不存在重合矩阵")
  print("- "*40)

  rectangles = [[31, 55, 19, 87], [17, 53, 98, 99], [34, 49, 11, 18], [9, 38, 22, 76], [77, 97, 81, 94]]
  print("矩阵集合为： ", rectangles)
  if is_overlap(rectangles):
    print("存在重合矩阵")
  else:
    print("不存在重合矩阵")

Overwriting ch14/vlsi_overlap_rectangles.py


In [0]:
!python3 -m ch14.vlsi_overlap_rectangles

矩阵集合为：  [[2, 5, 7, 44], [50, 53, 3, 82], [67, 84, 17, 62], [22, 38, 42, 50], [5, 7, 90, 95]]
不存在重合矩阵
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
矩阵集合为：  [[31, 55, 19, 87], [17, 53, 98, 99], [34, 49, 11, 18], [9, 38, 22, 76], [77, 97, 81, 94]]
存在重合矩阵


## 思考题

- <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200312162435.png width=800>

#### a
- 题目翻译的有问题，应该是其中一个区间的端点一定是最大重叠点，因为最大重叠点可能不止一个

- 证明：
  - 设 $m$ 是区间集合的一个最大重叠点，将 $m$ 沿着增大或减小的方向移动，则其第一次碰到的区间端点一定也是最大重叠点。所以区间集合中的一个端点一定是最大重叠点

#### b

- 基本思路
  - 以下述的形式表示区间
    - <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200312165136.png width=600>
  - 设 $y$ 是一条重直于坐标轴的线，将 $y$ 沿着 $x$ 轴增大的方向移动，如果在某一个 $x$ 点与 $y$ 重合的区间数最多，
  则说明 $x$ 是最大重叠点
***
- 具体实现
  - 取出所有区间的端点，组成集合 $X$， $X$ 中的元素按顺序排列依次表示为 $\{x_1, x_2, x_3, \cdots, x_{2n}\}$
  - 对于 $X$ 中的元素 $x_i$，如果其为左端点，则 $x_i.point = 1$，如果其为右端点，则$x_i.point = -1$
  - 包含端点 $x_i$ 的区间数目为 $num(i) = \sum_{j=1}^{i}x_j.point$
    - 此处假设如何区间的右端点与 $x_i$ 重合，则此区间不算重叠区间
      - 此假设最终得到的端点为某一区间的左端点
    - 求解最大重合端点等效于求解使得 $num(i)$ 取得最大值的 $i$
    - **对于一个区间的左端点与右端点重合的情况，如 $[17, 19]$ 与 $[19, 20]$ 应该先计算一个区间的左端点, 再计算另一个区间的右端点，否则会得到最大重叠只有 $1$ 的错误结论**
  - 为了求解最大重合端点，可将各个区间的端点依次插入至红黑树中，其中端点的关键字即为端点的值，同时端点还需要额外附加 $point$ 属性
    - **为了应对端点重合的情况，应该先插入区间的左端点，再插入右端点**
      - 在二叉搜索树中保证两个相等的值，后插入的值在先插入的值的右子树中
  - 红黑树的结点 $x$，还需要维护下述三个附加属性
    1. $all\_sum$
      - 以结点 $x$ 为根的子树中（包含结点 $x$） 的所有结点的 $point$ 属性之和
    2. $max\_sum$
      - 设以结点 $x$ 为根的子树中，将其所有的结点看成新的集合 $Y$， $max\_sum$ 即为此集合中 $num(i)$ 的最大值
    3. $point\_max$
      - $point\_max$ 为 $max\_sum$ 取得最大值对应的端点
  - 三个附加属性的维护
    1. $x.all\_sum = x.left.all\_sum + x.right.all\_sum + x.point$
    2. 
      - $ \left\{
      \begin{aligned}
      a &= x.left.max\_sum \\
      b &= x.left.all\_sum + x.point\\
      c &= b + x.right.max\_sum
      \end{aligned}
    \right.$
      - $ x.max\_sum = max(a, b, c)$
 
    3. $x.point\_max = \left\{
      \begin{aligned}
      &x.left.point\_max  && if \ x.max\_sum == a \\
      &x.key && if \ x.max\_sum == b \\
      &x.right.point\_max && if \ x.max\_sum == c
      \end{aligned}
      \right.
    $
    - $NIL$ 结点的 $all_sum$ 为 $0$, max\_sum 为 $-\infty$, $point\_max$ 为 $None$
  - 三个附加属性只与 $x, x.left, x.right$ 有关，由定理 14.1 可知，可在 $O(\lg n)$ 的时间内完成插入和删除操作
  - 将所有的端点插入扩展的红黑树中，则树根的 $point\_max$ 即为最大重叠点
***
- 代码实现

In [0]:
%%writefile ch14/get_max_overlap_points.py
"""获取最大区间的最大重叠点"""

from ch13.rb_tree import RBNode, RBTree


class RBNodeWithMaxOverlapPoints(RBNode):
  """含有相关附加属性的红黑树结点"""
  def __init__(self, all_sum=None, max_sum=None, point_max=None, point=None,**kwargs):
    self.all_sum = all_sum
    self.max_sum = max_sum
    self.point = point
    self.point_max = point_max
    super().__init__(**kwargs)


class RBTreeWithMaxOverlapPoints(RBTree):
  """含有相关附加属性的红黑树"""
  def __init__(self, keys=None):
    self.NIL = RBNodeWithMaxOverlapPoints(all_sum=0, max_sum=float('-inf'), point_max=None, color = RBNodeWithMaxOverlapPoints.B)
    self.root = self.NIL
    if keys:
      for key in keys:
        self.insert(key)

  @staticmethod
  def update(x):
    """更新结点 x 的相关属性"""
    x.all_sum = x.left.all_sum + x.point + x.right.all_sum
    b = x.left.all_sum + x.point
    c = b + x.right.max_sum
    x.max_sum = max(x.left.max_sum, b, c)
    if x.max_sum == x.left.max_sum:
      x.point_max = x.left.point_max
    elif x.max_sum == b:
      x.point_max = x.key
    else:
      x.point_max = x.right.point_max
    
  def insert(self, z):
    super().insert(z, fix=False)
    z.all_sum = z.point
    z.max_sum = z.point
    z.point_max = z.key
    x = z.parent
    while x != None:
      self.update(x)
      x = x.parent
    self.insert_fixup(z)
  
  def left_rotate(self, z):
    super().left_rotate(z)
    self.update(z)
    self.update(z.parent)

  def right_rotate(self, z):
    super().right_rotate(z)
    self.update(z)
    self.update(z.parent)
  
  def get_max_overlap_points(self):
    return None if self.root == None else self.root.point_max
  

def get_max_overlap_points(intervals):
  """借助扩展的数据结构查找区间的最大重叠点"""
  nodes_left = []
  nodes_right = []
  for interval in intervals:
    nodes_left.append(RBNodeWithMaxOverlapPoints(key=interval[0], point=1))
    nodes_right.append(RBNodeWithMaxOverlapPoints(key=interval[1], point=-1))
  rbt = RBTreeWithMaxOverlapPoints()
  nodes = nodes_left + nodes_right
  for node in nodes:
    rbt.insert(node)
  print("最大重叠点为：", rbt.get_max_overlap_points())
  print("最大重叠的区间数为：", rbt.root.max_sum)


if __name__ == "__main__":
  intervals = [(19, 20), (17, 19), (16, 21), (8, 9), (25, 30), (5, 8), (15, 23), (26, 26), (0, 3), (6, 10)]
  # intervals = [(17, 19), (19, 20)]
  get_max_overlap_points(intervals)

Writing ch14/get_max_overlap_points.py


In [0]:
!python3 -m ch14.get_max_overlap_points

最大重叠点为： 19
最大重叠的区间数为： 4


### 14-2
- <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200313081454.png width=800>

### a

- 可以采用双端队列，将各个元素依次入队，然后每次循环移动 $m-1$ 次，然后移出队首的元素。即可得到 Josephus
排列
- 时间复杂变为 $O(mn)$ 当 $m$ 为常数时，时间复杂度为 $O(n)$

In [0]:
%%writefile ch14/josephus.py
"""输出 josephus 排列"""

import collections
from ch14.os_tree import OSTree, OSTreeNode


def josephus_deque(n, m):
  """借助队列求解 josephus 排列"""
  dq = collections.deque(range(1, n+1))
  res = []
  while len(dq) > 0:
    dq.rotate(-(m-1))
    res.append(dq.popleft())
  return res

Overwriting ch14/josephus.py


In [0]:
import ch14.josephus

In [0]:
n = 7
m = 3
print("({}, {}) 的 Josephus 排列为 {}".format(n, m, ch14.josephus.josephus_deque(n, m)))

(7, 3) 的 Josephus 排列为 [3, 6, 2, 7, 5, 1, 4]


#### b

- $m$ 不为常数时，需要借助顺序统计树和求余操作，如此可在 $O(n\lg n)$ 的时间内求出 $(n, m)$ 的 
Josephus 排列

In [0]:
%%writefile -a ch14/josephus.py



def josephus_ostree(n, m):
  """借助顺序统树计算 josephus 排列"""
  ostree = OSTree(keys=range(1, n+1))
  size = n
  index = 0
  res = []
  while size > 0:
    index = (index + m) % (size)
    if index == 0:
      index = size
    target = ostree.select(index)
    ostree.delete(target)
    res.append(target.key)
    size -= 1
    index -= 1
  return res


Appending to ch14/josephus.py


In [0]:
imp.reload(ch14.josephus)

<module 'ch14.josephus' from '/content/drive/My Drive/Colab Notebooks/CLRS/CLRS_notes/ch14/josephus.py'>

In [0]:
n = 7
m = 3
print("({}, {}) 的 Josephus 排列为 {}".format(n, m, ch14.josephus.josephus_ostree(n, m)))

(7, 3) 的 Josephus 排列为 [3, 6, 2, 7, 5, 1, 4]
